# TODO

- Fix the timing out so I can add everything to the sqlite db.
- Finish grabbing the data for year 2017.
- Train IMDB sentiment analysis model from in the book below.
- Use that model and classify all the post titles, and comments.
- Use that with ../cryptocurrencies/ data and look for correlations between volume or returns


## references for later
- https://livebook.manning.com/#!/book/real-world-machine-learning/chapter-8/1
- http://praw.readthedocs.io/en/latest/tutorials/comments.html
- https://github.com/crazyguitar/pysheeet/blob/master/docs/notes/python-sqlalchemy.rst

In [1]:
import praw
from praw.models import MoreComments
from prawcore.exceptions import PrawcoreException

from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

import pandas as pd

reddit = praw.Reddit(client_id='MZaMr8n8Ds6XMg',
                     client_secret='bxJOAK7UID5MtnKVhPxqoJTuD-U',
                     password='mecp26',
                     user_agent='crypto stuff by /u/Picklebillharvey',
                     username='Picklebillharvey')

print(reddit.user.me())

def make_tables():
    engine.execute('CREATE TABLE IF NOT EXISTS "Posts" ('
                   'id TEXT NOT NULL,'
                   'author TEXT, '
                   'domain TEXT,'
                   'created_utc INTEGER,'
                   'title TEXT,'
                   'view_count INTEGER,'
                   'ups INTEGER,'
                   'downs INTEGER,'
                   'score INTEGER,'
                   'edited BOOL,'
                   'num_crossposts INTEGER,'
                   'num_reports INTEGER,'
                   'banned_utc INTEGER,'
                   'removal_reason TEXT,'
                   'num_comments INTEGER,'
                   'pinned BOOL,'
                   'subreddit TEXT,'
                   'PRIMARY KEY (id));')


    engine.execute('CREATE TABLE IF NOT EXISTS "Comments" ('
                   'id TEXT NOT NULL,'
                   'author TEXT, '
                   'permalink TEXT,'
                   'created_utc INTEGER,'
                   'edited BOOL,'
                   'stickied BOOL,'
                   'ups INTEGER,'
                   'downs INTEGER,'
                   'score INTEGER,'
                   'removal_reason TEXT,'
                   'num_reports INTEGER,'
                   'parent TEXT,'
                   'subreddit TEXT,'
                   'body TEXT,'
                   'PRIMARY KEY (permalink));'
                  )

def insert_into_posts(values):
    engine.execute('INSERT OR IGNORE INTO "Posts" '
           '(\
           id, \
           author, \
           domain, \
           created_utc, \
           title, \
           view_count, \
           ups, \
           downs, \
           score, \
           edited, \
           num_crossposts, \
           num_reports, \
           banned_utc, \
           removal_reason, \
           num_comments, \
           pinned, \
           subreddit \
           ) '

           'VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', 
            (values))

def insert_into_comments(values):
    engine.execute('INSERT OR IGNORE INTO "Comments" '
           '(\
           id, \
           author, \
           permalink, \
           created_utc, \
           ups, \
           downs, \
           score, \
           edited, \
           removal_reason, \
           num_reports, \
           parent, \
           stickied, \
           subreddit, \
           body \
           ) '

           'VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)', 
            (values))
    
def get_post_values(submission):
    ID = submission.id
    author = str(submission.author)
    domain = str(submission.domain)
    created_utc = int(submission.created_utc or 0)
    title = submission.title
    view_count = int(submission.view_count or 0)
    ups = int(submission.ups or 0)
    downs = int(submission.downs or 0)
    score = int(submission.score or 0)
    edited = submission.edited
    num_crossposts = int(submission.num_crossposts or 0)
    num_reports = int(submission.num_reports or 0)
    banned_utc = int(submission.banned_at_utc or 0)
    removal_reason = str(submission.removal_reason)
    num_comments = int(submission.num_comments or 0)
    pinned = submission.pinned
    subreddit = str(submission.subreddit)

    print pd.to_datetime(created_utc, unit='s'), ID, subreddit

    values = (ID, author, domain, created_utc, title, view_count, 
              ups, downs, score, edited, num_crossposts,
              num_reports, banned_utc, removal_reason,
              
              num_comments, pinned, subreddit)

    return values



def get_comment_values(comment):
    author = str(comment.author)
    ID = str(comment.id)
    permalink = comment.permalink
    created_utc = int(comment.created_utc)
    edited = comment.edited
    ups = int(comment.ups)
    downs = int(comment.downs)
    score = int(comment.score)
    removal_reason = str(comment.removal_reason)
    num_reports = int(comment.num_reports or 0)
    parent = str(comment.parent())
    stickied = comment.stickied
    subreddit = str(comment.subreddit_name_prefixed)
    body = comment.body

    values = (author, ID, permalink, created_utc, edited,
              ups, downs, score, removal_reason, num_reports,
              parent, stickied, subreddit, body)

    return values

def get_subreddit_data(subreddit, date_start_utc, date_end_utc):
    for submission in subreddit.submissions(date_start_utc, date_end_utc): 
        values = get_post_values(submission)            
        insert_into_posts(values)

        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            values = get_comment_values(comment)
            insert_into_comments(values)
            
        

Picklebillharvey


In [ ]:
## !rm db.sqlite
engine = create_engine("sqlite:///db.sqlite")
conn = engine.connect()

conn.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')

make_tables()

df = pd.DataFrame(pd.read_csv('crypto_subreddits.txt', header=None))

df.columns = ['subs','cointype']
df.iloc[:,0] = df.iloc[:,0].str.replace('/','')

bitcoin_subs = df.iloc[9:12,:].reset_index(drop=True).loc[:,'subs'].values

day_utc = 86400

start_date = 1483228800

date_start_utc = start_date
date_end_utc = start_date + day_utc

print bitcoin_subs

for sub in bitcoin_subs:
    print '----------------------------------------'
    print '----------------------------------------'
    print '----------------------------------------'
    print sub
    print '----------------------------------------'
    print '----------------------------------------'
    print '----------------------------------------'
    while date_end_utc <= 1514764800:
        
        print '----------------------------------------'
        print pd.to_datetime(date_start_utc, unit='s'), pd.to_datetime(date_end_utc, unit='s')
        print '----------------------------------------'
        
        subreddit = reddit.subreddit(sub)
        try:
            get_subreddit_data(subreddit, date_start_utc, date_end_utc)

        except PrawcoreException:
            print 'timeout...try again...'
            get_subreddit_data(subreddit, date_start_utc, date_end_utc)
        
        date_start_utc += day_utc
        date_end_utc += day_utc
    
    date_start_utc = start_date
    date_end_utc = start_date + day_utc



['ethtrader' 'ethereum' 'ethermining']
----------------------------------------
----------------------------------------
----------------------------------------
ethtrader
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
2017-01-01 00:00:00 2017-01-02 00:00:00
----------------------------------------
2017-01-01 23:38:00 5lhydb ethtrader
2017-01-01 22:28:32 5lhlpr ethtrader
2017-01-01 20:51:50 5lh3ao ethtrader
2017-01-01 19:31:26 5lgok3 ethtrader
2017-01-01 18:48:30 5lggjb ethtrader
2017-01-01 16:43:58 5lfute ethtrader
2017-01-01 15:15:38 5lfhrz ethtrader
2017-01-01 15:14:34 5lfhm9 ethtrader
2017-01-01 08:17:47 5leajf ethtrader
2017-01-01 07:06:40 5le2oz ethtrader
----------------------------------------
2017-01-02 00:00:00 2017-01-03 00:00:00
----------------------------------------
2017-01-02 23:01:13 5lo8va ethtrader
2017-01-02 22:19:53 5lo0vt ethtrader
2017-01-02 19:36:

2017-01-10 02:08:46 5n2617 ethtrader
2017-01-10 02:05:45 5n25fb ethtrader
----------------------------------------
2017-01-11 00:00:00 2017-01-12 00:00:00
----------------------------------------
2017-01-11 23:04:40 5nfjmu ethtrader
2017-01-11 23:01:17 5nfiy7 ethtrader
2017-01-11 21:03:37 5netbs ethtrader
2017-01-11 19:45:04 5nebms ethtrader
2017-01-11 16:45:02 5nd77i ethtrader
2017-01-11 16:22:26 5nd2fl ethtrader
2017-01-11 15:51:47 5ncvzn ethtrader
2017-01-11 13:27:09 5nc5ln ethtrader
2017-01-11 13:09:53 5nc2sh ethtrader
2017-01-11 12:59:57 5nc169 ethtrader
2017-01-11 12:08:07 5nbtna ethtrader
2017-01-11 07:05:43 5naro1 ethtrader
2017-01-11 05:56:18 5naheh ethtrader
2017-01-11 04:20:58 5na1i1 ethtrader
2017-01-11 03:41:26 5n9ue4 ethtrader
2017-01-11 03:40:26 5n9u8b ethtrader
2017-01-11 03:12:21 5n9p64 ethtrader
----------------------------------------
2017-01-12 00:00:00 2017-01-13 00:00:00
----------------------------------------
2017-01-12 21:30:31 5nm629 ethtrader
2017-01-12 16:05

2017-01-22 19:59:21 5pj7kv ethtrader
2017-01-22 15:06:35 5phd70 ethtrader
2017-01-22 15:06:28 5phd6c ethtrader
2017-01-22 14:39:54 5ph8a6 ethtrader
2017-01-22 14:33:50 5ph76e ethtrader
2017-01-22 11:44:00 5pgj75 ethtrader
2017-01-22 07:05:25 5pfrkn ethtrader
2017-01-22 01:57:52 5pegii ethtrader
2017-01-22 01:37:22 5pecrb ethtrader
----------------------------------------
2017-01-23 00:00:00 2017-01-24 00:00:00
----------------------------------------
2017-01-23 21:17:37 5prqdu ethtrader
2017-01-23 20:42:46 5pride ethtrader
2017-01-23 20:15:01 5prc4r ethtrader
2017-01-23 20:08:35 5praon ethtrader
2017-01-23 19:53:50 5pr6z2 ethtrader
2017-01-23 19:47:34 5pr5g4 ethtrader
2017-01-23 16:07:35 5ppn15 ethtrader
2017-01-23 15:01:28 5pp7ji ethtrader
2017-01-23 14:48:58 5pp51d ethtrader
2017-01-23 12:23:47 5pod65 ethtrader
2017-01-23 12:20:13 5pocl5 ethtrader
2017-01-23 07:07:25 5pn2l1 ethtrader
2017-01-23 05:36:43 5pmmmz ethtrader
2017-01-23 01:51:35 5plexy ethtrader
2017-01-23 01:25:53 5pl9sb 

2017-02-03 19:34:59 5rw0kk ethtrader
2017-02-03 18:44:36 5rvpkt ethtrader
2017-02-03 17:47:13 5rvd1f ethtrader
2017-02-03 17:29:07 5rv8z3 ethtrader
2017-02-03 16:46:29 5ruzg0 ethtrader
2017-02-03 15:21:33 5ruha8 ethtrader
2017-02-03 15:16:00 5rug6e ethtrader
2017-02-03 15:01:56 5rudeg ethtrader
2017-02-03 11:04:20 5rtbqq ethtrader
2017-02-03 10:36:46 5rt7xw ethtrader
2017-02-03 08:17:52 5rsqw4 ethtrader
2017-02-03 07:06:05 5rsil6 ethtrader
2017-02-03 05:02:41 5rs15k ethtrader
2017-02-03 03:04:57 5rrhre ethtrader
2017-02-03 02:49:05 5rreww ethtrader
----------------------------------------
2017-02-04 00:00:00 2017-02-05 00:00:00
----------------------------------------
2017-02-04 23:44:07 5s47d3 ethtrader
2017-02-04 22:55:02 5s3vih ethtrader
2017-02-04 21:36:33 5s3bxc ethtrader
2017-02-04 19:49:42 5s2luw ethtrader
2017-02-04 18:25:30 5s24h8 ethtrader
2017-02-04 18:07:14 5s20uo ethtrader
2017-02-04 17:58:23 5s1z0q ethtrader
2017-02-04 13:45:02 5s0mc5 ethtrader
2017-02-04 07:08:15 5rzcf7 

2017-02-13 17:49:14 5tub9s ethtrader
2017-02-13 17:40:56 5tu9a3 ethtrader
2017-02-13 16:03:38 5ttnht ethtrader
2017-02-13 15:11:38 5ttcl6 ethtrader
2017-02-13 14:57:53 5tt9ys ethtrader
2017-02-13 14:00:30 5tszka ethtrader
2017-02-13 13:28:15 5tsuhh ethtrader
2017-02-13 13:07:58 5tsrbg ethtrader
2017-02-13 12:15:53 5tsjk4 ethtrader
2017-02-13 10:40:06 5ts7t2 ethtrader
2017-02-13 10:13:49 5ts4w2 ethtrader
2017-02-13 10:09:13 5ts4dn ethtrader
2017-02-13 07:42:29 5tro7a ethtrader
2017-02-13 07:07:09 5trjwy ethtrader
2017-02-13 06:09:03 5trccp ethtrader
2017-02-13 03:39:36 5tqo8d ethtrader
2017-02-13 02:56:47 5tqgzx ethtrader
2017-02-13 00:21:21 5tpmrx ethtrader
----------------------------------------
2017-02-14 00:00:00 2017-02-15 00:00:00
----------------------------------------
2017-02-14 23:47:39 5u3yz6 ethtrader
2017-02-14 23:27:02 5u3uw9 ethtrader
2017-02-14 23:03:37 5u3q3t ethtrader
2017-02-14 20:08:57 5u2nos ethtrader
2017-02-14 20:06:57 5u2n9f ethtrader
2017-02-14 19:18:07 5u2brl 

...

2017-03-02 23:53:49 5x6y8i ethtrader
2017-03-02 23:31:40 5x6txl ethtrader
2017-03-02 23:16:54 5x6r0x ethtrader
2017-03-02 23:03:11 5x6o5v ethtrader
2017-03-02 22:57:53 5x6n06 ethtrader
2017-03-02 22:30:51 5x6he6 ethtrader
2017-03-02 22:04:20 5x6bm1 ethtrader
2017-03-02 22:03:54 5x6biq ethtrader
2017-03-02 21:54:45 5x69ir ethtrader
2017-03-02 20:40:38 5x5srs ethtrader
2017-03-02 19:56:02 5x5is8 ethtrader
2017-03-02 19:51:12 5x5hrc ethtrader
2017-03-02 19:30:18 5x5cza ethtrader
2017-03-02 19:29:25 5x5csc ethtrader
2017-03-02 19:23:18 5x5bef ethtrader
2017-03-02 19:21:12 5x5awy ethtrader
2017-03-02 19:04:08 5x56uv ethtrader
2017-03-02 18:51:08 5x53qn ethtrader
2017-03-02 18:42:44 5x51oh ethtrader
2017-03-02 18:33:23 5x4zip ethtrader
2017-03-02 18:33:14 5x4zgr ethtrader
2017-03-02 18:07:04 5x4tbo ethtrader
2017-03-02 17:57:32 5x4r2w ethtrader
2017-03-02 17:51:06 5x4pno ethtrader
2017-03-02 17:12:27 5x4gxx ethtrader
2017-03-02 17:01:28 5x4ect ethtrader
2017-03-02 16:53:16 5x4cce ethtrader
2

2017-03-07 11:04:35 5xzr9t ethtrader
2017-03-07 10:44:43 5xzp0c ethtrader
2017-03-07 10:00:11 5xzjwv ethtrader
2017-03-07 07:23:52 5xz2jv ethtrader
2017-03-07 07:04:59 5xz05e ethtrader
2017-03-07 06:58:57 5xyzc3 ethtrader
2017-03-07 06:38:23 5xywn5 ethtrader
2017-03-07 06:33:23 5xyvxf ethtrader
2017-03-07 05:07:32 5xyjiw ethtrader
2017-03-07 04:27:46 5xyczh ethtrader
2017-03-07 03:25:45 5xy279 ethtrader
2017-03-07 00:31:52 5xx5qg ethtrader
----------------------------------------
2017-03-08 00:00:00 2017-03-09 00:00:00
----------------------------------------
2017-03-08 23:21:33 5ybgba ethtrader
2017-03-08 22:05:10 5yazi9 ethtrader
2017-03-08 21:45:19 5yav2p ethtrader
2017-03-08 19:45:07 5ya4du ethtrader
2017-03-08 18:28:39 5y9mvf ethtrader
2017-03-08 18:25:31 5y9m6u ethtrader
2017-03-08 16:49:14 5y902a ethtrader
2017-03-08 16:48:40 5y8zyg ethtrader
2017-03-08 16:08:45 5y8r4p ethtrader
2017-03-08 16:04:47 5y8q99 ethtrader
2017-03-08 15:54:46 5y8o2g ethtrader
2017-03-08 14:19:26 5y84wd 

2017-03-13 15:59:25 5z5sgb ethtrader
2017-03-13 15:55:44 5z5rpc ethtrader
2017-03-13 15:50:35 5z5qly ethtrader
2017-03-13 15:48:24 5z5q53 ethtrader
2017-03-13 15:39:48 5z5o6a ethtrader
2017-03-13 15:38:49 5z5nyo ethtrader
2017-03-13 15:37:29 5z5nnq ethtrader
2017-03-13 15:36:12 5z5ndv ethtrader
2017-03-13 15:35:01 5z5n5f ethtrader
2017-03-13 15:33:51 5z5mwv ethtrader
2017-03-13 15:30:28 5z5m8f ethtrader
2017-03-13 15:26:40 5z5lfq ethtrader
2017-03-13 15:23:02 5z5kpe ethtrader
2017-03-13 15:14:57 5z5j20 ethtrader
2017-03-13 15:08:33 5z5hnp ethtrader
2017-03-13 15:00:11 5z5fva ethtrader
2017-03-13 15:00:09 5z5fv4 ethtrader
2017-03-13 14:49:28 5z5dsw ethtrader
2017-03-13 14:49:16 5z5dr8 ethtrader
2017-03-13 14:33:48 5z5akh ethtrader
2017-03-13 14:23:47 5z58p2 ethtrader
2017-03-13 14:16:09 5z578j ethtrader
2017-03-13 14:14:00 5z56u7 ethtrader
2017-03-13 14:10:16 5z564c ethtrader
2017-03-13 13:48:37 5z5254 ethtrader
2017-03-13 13:42:08 5z5115 ethtrader
2017-03-13 13:34:53 5z4zr1 ethtrader
2

2017-03-15 13:29:38 5zjca9 ethtrader
2017-03-15 13:28:23 5zjc13 ethtrader
2017-03-15 13:27:41 5zjbwc ethtrader
2017-03-15 12:39:04 5zj3eh ethtrader
2017-03-15 12:17:27 5zj03d ethtrader
2017-03-15 12:15:30 5zizrv ethtrader
2017-03-15 11:57:36 5zix0h ethtrader
2017-03-15 11:22:26 5zirm3 ethtrader
2017-03-15 10:10:08 5ziie4 ethtrader
2017-03-15 09:08:04 5zib4n ethtrader
2017-03-15 09:02:00 5ziaft ethtrader
2017-03-15 08:44:58 5zi8mq ethtrader
2017-03-15 08:32:49 5zi7dg ethtrader
2017-03-15 07:36:35 5zi12u ethtrader
2017-03-15 07:31:50 5zi0iz ethtrader
2017-03-15 07:05:35 5zhx6j ethtrader
2017-03-15 06:13:38 5zhqq0 ethtrader
2017-03-15 05:06:04 5zhh5i ethtrader
2017-03-15 04:00:15 5zh78b ethtrader
2017-03-15 03:04:57 5zgy52 ethtrader
2017-03-15 02:32:46 5zgsfn ethtrader
2017-03-15 01:52:33 5zgkrq ethtrader
2017-03-15 01:21:36 5zget3 ethtrader
2017-03-15 01:18:49 5zge94 ethtrader
2017-03-15 00:14:15 5zg1n0 ethtrader
----------------------------------------
2017-03-16 00:00:00 2017-03-17 00:

2017-03-17 14:29:32 5zxvue ethtrader
2017-03-17 14:23:42 5zxuoj ethtrader
2017-03-17 14:18:06 5zxtk7 ethtrader
2017-03-17 14:07:34 5zxrjk ethtrader
2017-03-17 13:45:31 5zxndo ethtrader
2017-03-17 13:13:49 5zxhrs ethtrader
2017-03-17 13:03:34 5zxg17 ethtrader
2017-03-17 12:56:16 5zxero ethtrader
2017-03-17 12:52:35 5zxe53 ethtrader
2017-03-17 12:36:05 5zxbem ethtrader
2017-03-17 12:29:30 5zxabw ethtrader
2017-03-17 12:28:26 5zxa5a ethtrader
2017-03-17 12:24:51 5zx9lj ethtrader
2017-03-17 12:16:31 5zx8dv ethtrader
2017-03-17 12:10:11 5zx7es ethtrader
2017-03-17 12:03:29 5zx6f5 ethtrader
2017-03-17 11:28:03 5zx16d ethtrader
2017-03-17 11:20:15 5zx03z ethtrader
2017-03-17 11:19:38 5zx01k ethtrader
2017-03-17 11:11:14 5zwyv8 ethtrader
2017-03-17 10:52:54 5zwwgu ethtrader
2017-03-17 10:36:42 5zwui6 ethtrader
2017-03-17 10:14:27 5zwrvt ethtrader
2017-03-17 10:02:25 5zwqc1 ethtrader
2017-03-17 10:00:51 5zwq35 ethtrader
2017-03-17 09:36:07 5zwn98 ethtrader
2017-03-17 09:35:07 5zwn5h ethtrader
2